In [1]:
import logging
import random
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [2]:
"""
Profiles of filenames.
    PROVINCES_JSON_FILENAME: filename of provinces_df saved in json format
    MAPBOX_ACCESS_TOKEN: access token of mapbox
    HTML_FILENAME: filename of output html file
    LOG_FILENAME: filename of logging file
"""

PROVINCES_JSON_FILENAME = 'provinces.json'
MAPBOX_ACCESS_TOKEN = 'pk.eyJ1IjoibGlzdGVuemNjIiwiYSI6ImNrMzU5MmpxZDAxMXEzbXQ0dnd4YTZ2NDAifQ.GohcgYXFsbDqfsi_7SXdpA'
HTML_FILENAME = 'ncov_map.html'
LOG_FILENAME = "lastlog.log"

# initialize logging file
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
with open(LOG_FILENAME, 'w') as f:
    f.writelines([LOG_FORMAT, '\n'])
logging.basicConfig(filename=LOG_FILENAME, level=logging.DEBUG, format=LOG_FORMAT)

In [3]:
"""
Fetch provinces_df from json file.
    provinces_df: dataframe of counts, index as provinceName
    main_title: the title of the graph showing global numbers
"""

provinces_df = pd.read_json(PROVINCES_JSON_FILENAME)
provinces_df = provinces_df.set_index('provinceName', drop=False)
provinces_df

main_title = '全国ncov {}例'.format(provinces_df.confirmedCount.sum())

In [4]:
"""
Profiles of scatter traces.
    colorscale: instance of colormap
    showscale: if show the colorbar
    size: marker size
    visible: False means initializing the scatter traces as invisible, True means visible
"""

colorscale = px.colors.carto.Redor
showscale = False
size = 9
visible = [True, True]

In [5]:
"""
Setup scatter traces.
    scatter_traces: a dict in format of {{provinceName}: {scattermapbox instance}}
    provinceNames: name list of provinces
"""

scatter_traces = dict()
provinceNames = provinces_df.index.unique()

# for each province
# 1. fetch its cities
# 2. setup its scatter
# 3. save the scatter into scatter_traces
log_confirmedCount = np.log10(provinces_df.confirmedCount.values + 1)
cmax, cmin = log_confirmedCount.max(), log_confirmedCount.min()
for prov in provinceNames:
    print(prov, end=', ')
    
    # fetch cities
    cities = provinces_df.loc[prov]
    # solution of innormal situation
    # if there is only one city in the province,
    # cities will be a pd.Series rather than pd.DataFrame,
    # so it should be converted.
    if isinstance(cities, pd.Series):
        cities = pd.DataFrame(cities).T
    
    # setup scatter
    # prepare marker for each city
    marker = go.scattermapbox.Marker(
        color=np.log10(cities['confirmedCount'].astype(np.float)+1),
        size=size,
        colorscale=colorscale,
        cmax=cmax,
        cmin=cmin,
        showscale=showscale)
    # setup scatter
    scatter = go.Scattermapbox(
        lat=cities['lat'],
        lon=cities['lng'],
        text=cities.provinceName + '-' + cities.cityName + '-' + cities.confirmedCount.astype(str),
        mode='markers',
        marker=marker,
        name='{} {}例'.format(prov, cities.confirmedCount.sum()),
        visible=random.choice(visible), # randomly choose visible state
    )
    
    # save the scatter
    scatter_traces[prov] = scatter

print('done.')

湖北省, 广东省, 浙江省, 河南省, 湖南省, 安徽省, 江西省, 江苏省, 重庆市, 山东省, 四川省, 北京市, 黑龙江省, 上海市, 福建省, 河北省, 广西壮族自治区, 陕西省, 云南省, 海南省, 山西省, 辽宁省, 贵州省, 天津市, 甘肃省, 吉林省, 内蒙古自治区, 宁夏回族自治区, 新疆维吾尔自治区, 青海省, 西藏自治区, done.


In [6]:
"""
Pair button for each trace.
    scatter_buttons: buttons paired with scatter_traces
"""
scatter_buttons = []


def one_hot(trg, lst=provinceNames):
    # Get one hot presention of trg in lst
    # if trg is None, return all true
    if trg is None:
        return [True for e in lst]
    # return one hot
    return [e==trg for e in lst]


# Add button to show all provinces
visible = one_hot(None)
title = main_title
label = 'ALL'
scatter_buttons.append(dict(
    label=label,
    method='update',
    args=[dict(visible=visible),
          dict(title=title)]
))

# Add button for each province
for prov in provinceNames:
    visible = one_hot(prov)
    title = scatter_traces[prov].name
    label = prov
    scatter_buttons.append(dict(
        label=label,
        method='update',
        args=[dict(visible=visible),
              dict(title=title)]
    ))

print('Done.')

Done.


In [13]:
fig = go.Figure()

for prov in provinceNames:
    print(prov, end=', ')
    fig.add_trace(scatter_traces[prov])
print('done.')

mapbox = go.layout.Mapbox(
    accesstoken=MAPBOX_ACCESS_TOKEN,
    bearing=0,
    pitch=0,
    zoom=2,
    center=go.layout.mapbox.Center(
        lat=provinces_df.lat.mean(),
        lon=provinces_df.lng.mean()))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=mapbox,
    clickmode='event',
    title_text=main_title,
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=scatter_buttons
    )],
)

fig.write_html(HTML_FILENAME)

fig.show()

湖北省, 广东省, 浙江省, 河南省, 湖南省, 安徽省, 江西省, 江苏省, 重庆市, 山东省, 四川省, 北京市, 黑龙江省, 上海市, 福建省, 河北省, 广西壮族自治区, 陕西省, 云南省, 海南省, 山西省, 辽宁省, 贵州省, 天津市, 甘肃省, 吉林省, 内蒙古自治区, 宁夏回族自治区, 新疆维吾尔自治区, 青海省, 西藏自治区, done.


In [21]:
fig = plotly.subplots.make_subplots(
    rows=3, cols=1,
    shared_xaxes=False,
    #vertical_spacing=0.03,
    specs=[[{"type": "bar"}],
           [{"type": "table"}],
           [{"type": "mapbox"}]]
)

df = provinces_df.loc['北京市']

fig.add_trace(
    go.Table(
        header=dict(
            values=df.columns,
            font=dict(size=10),
            align="left"),
        cells=dict(
            values=[df[k].tolist() for k in df.columns],
            align = "left")
    ),
    row=2, col=1
)

fig.add_trace(
    go.Bar(x=df.cityName, y=df.confirmedCount),
    row=1, col=1
)

# fig.add_trace(
#     go.Scattergeo(lat=df["lat"],
#                   lon=df["lng"],
#                   mode="markers",
#                   hoverinfo="text",
#                   showlegend=False,
#                   marker=dict(color="crimson", size=4, opacity=0.8)),
#     row=3, col=1
# )

fig.update_layout(
    autosize=True,
    hovermode='closest',
    clickmode='event',
    title_text='北京市'
)

fig.add_trace(scatter_traces[prov])

fig.update_mapboxes(
    accesstoken=MAPBOX_ACCESS_TOKEN,
    bearing=0,
    pitch=0,
    zoom=2,
    center=go.layout.mapbox.Center(
        lat=provinces_df.lat.mean(),
        lon=provinces_df.lng.mean())
)

fig.show()

In [18]:
dir(fig)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_annotation_like',
 '_animation_duration_validator',
 '_animation_easing_validator',
 '_batch_layout_edits',
 '_batch_trace_edits',
 '_bracket_re',
 '_build_dispatch_plan',
 '_build_update_params_from_batch',
 '_config',
 '_data',
 '_data_defaults',
 '_data_objs',
 '_data_validator',
 '_dispatch_layout_change_callbacks',
 '_dispatch_trace_change_callbacks',
 '_frame_objs',
 '_frames_validator',
 '_get_child_prop_defaults',
 '_get_child_props',
 '_grid_ref',
 '_grid_str',
 '_in_batch_mode',
 '_index_is',
 '_init_child_props',
 '_initialize_layout_template',

In [9]:
tab = ff.create_table(provinces_df.loc['北京市'])
tab.show()

In [8]:
tab.update_layout(
    title_text='北京市',
)
tab.layout.margin.update({'t':50, 'b':100})
tab.show()
